In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
df = pd.read_csv("../input/fake-news/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
df = df.dropna()

In [5]:
x= df.drop(['label'],axis=1)
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [6]:
y = df.label

In [7]:
x.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [10]:
# vocublary size
voc_size=5000

One Hot Representation

In [11]:
messages = df.copy()
messages.reset_index(inplace=True)
messages.head(1)

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1


In [12]:
import nltk
import re
from nltk.corpus import stopwords

Data Preprocessing

In [13]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(words) for words in review if not words in stopwords.words("english")]
    review = " ".join(review)
    corpus.append(review)

In [14]:
one_hot_repr = [one_hot(word, voc_size) for word in corpus]
one_hot_repr

[[574, 472, 3412, 2940, 3767, 2329, 2451, 1066, 2596, 4856],
 [4535, 4115, 727, 2181, 4072, 1061, 4456],
 [3686, 3387, 4544, 4484],
 [1181, 2573, 2988, 3363, 4646, 2279],
 [3711, 4072, 4094, 1372, 3398, 567, 4072, 4966, 1033, 3108],
 [4392,
  3169,
  4101,
  2399,
  4270,
  4197,
  1788,
  3980,
  1753,
  4821,
  3024,
  2110,
  822,
  1139,
  4456],
 [3064, 1708, 17, 267, 3642, 3874, 3515, 4168, 3451, 2600, 3741],
 [2867, 706, 4806, 3121, 2502, 805, 4197, 2290, 3451, 2600, 3741],
 [1217, 4664, 1436, 3828, 3019, 525, 2360, 820, 4197, 1230],
 [4501, 2366, 568, 3679, 3754, 832, 185, 2916],
 [2752, 3495, 2436, 389, 3865, 4640, 970, 3180, 930, 790, 2352],
 [3363, 2809, 3767, 525, 4197, 2502],
 [3696, 1655, 2392, 4336, 3225, 3512, 4050, 3598, 4438],
 [980, 24, 2147, 4452, 3098, 4173, 1046, 3451, 2600, 3741],
 [4561, 551, 3010, 2781, 2910, 3451, 2600, 3741],
 [4422, 2179, 694, 468, 4717, 577, 2624, 2794, 3873, 4570],
 [3206, 4488, 4115],
 [2281, 2822, 3289, 1609, 4197, 2424, 3904, 4456],
 [3

# Embedding Representation

In [15]:
sent_len = 20
embedding_doc = pad_sequences(one_hot_repr, padding='pre',maxlen=sent_len)
embedding_doc

array([[   0,    0,    0, ..., 1066, 2596, 4856],
       [   0,    0,    0, ..., 4072, 1061, 4456],
       [   0,    0,    0, ..., 3387, 4544, 4484],
       ...,
       [   0,    0,    0, ..., 3451, 2600, 3741],
       [   0,    0,    0, ...,  650,  631, 3694],
       [   0,    0,    0, ..., 2204, 1515,  747]], dtype=int32)

# Creating Model

In [16]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_len ))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2022-03-21 06:46:36.199977: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
x_final = np.array(embedding_doc)
y_final = np.array(y)

In [18]:
from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(x_final, y_final, random_state=0, test_size=0.2)

In [19]:
model.fit(x_train, y_train , validation_data =(x_test, y_test), epochs=10, batch_size=64)

2022-03-21 06:46:37.081963: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
229/229 [==============================] - 10s 31ms/step - loss: 0.3191 - accuracy: 0.8583 - val_loss: 0.1849 - val_accuracy: 0.9193
Epoch 2/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1396 - accuracy: 0.9449 - val_loss: 0.1940 - val_accuracy: 0.9191
Epoch 3/10
229/229 [==============================] - 7s 28ms/step - loss: 0.1029 - accuracy: 0.9638 - val_loss: 0.2089 - val_accuracy: 0.9188
Epoch 4/10
229/229 [==============================] - 7s 29ms/step - loss: 0.0737 - accuracy: 0.9746 - val_loss: 0.2176 - val_accuracy: 0.9196
Epoch 5/10
229/229 [==============================] - 7s 32ms/step - loss: 0.0504 - accuracy: 0.9839 - val_loss: 0.2447 - val_accuracy: 0.9191
Epoch 6/10
229/229 [==============================] - 7s 29ms/step - loss: 0.0327 - accuracy: 0.9899 - val_loss: 0.2834 - val_accuracy: 0.9098
Epoch 7/10
229/229 [==============================] - 7s 29ms/step - loss: 0.0241 - accuracy: 0.9932 - val_loss: 0.3623 - val_accuracy: 0.915

# Accuracy and confusion matrix

In [20]:
y_pred = model.predict(x_test)
y_pred[1]
classes_x=np.argmax(y_pred,axis=1)
classes_x

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix
score = accuracy_score(y_test, classes_x)
score


0.5578342904019689

In [22]:
cm = confusion_matrix(y_test, classes_x)
cm

array([[2040,    0],
       [1617,    0]])

# Adding Dropout

In [23]:
from tensorflow.keras.layers import Dropout
#creating model
embedding_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_features, input_length = sent_len))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])


# Accuracy and Confusion Matrix

In [24]:
y_pred = model1.predict(x_test)
y_pred[1]
class_x=np.argmax(y_pred,axis=1)
class_x

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix
score = accuracy_score(y_test, class_x)
score

0.5578342904019689